# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

Here the patience are initliazed, 

In [71]:
patient = [
    {'ge40':True, 'it40':False, 'premeno':False, 'Deg-malig-3':True, 'Deg-malig-2':False, 'Deg-malig-1':False, 'Inv-nodes 0-2': False, 'Inv-nodes 3-5': True, 'Inv-nodes 6-8': False},
    {'ge40':False, 'it40':True, 'premeno':False, 'Deg-malig-3':True, 'Deg-malig-2':False, 'Deg-malig-1':False, 'Inv-nodes 0-2': True, 'Inv-nodes 3-5': False, 'Inv-nodes 6-8': False},
    {'ge40':True, 'it40':False, 'premeno':False, 'Deg-malig-3':True, 'Deg-malig-2':False, 'Deg-malig-1':False, 'Inv-nodes 0-2': False, 'Inv-nodes 3-5': False, 'Inv-nodes 6-8': True},
    {'ge40':True, 'it40':False, 'premeno':False, 'Deg-malig-3': False, 'Deg-malig-2':True, 'Deg-malig-1':False, 'Inv-nodes 0-2': True, 'Inv-nodes 3-5': False, 'Inv-nodes 6-8': False},
    {'ge40':False, 'it40':False, 'premeno':True, 'Deg-malig-3': True, 'Deg-malig-2':False, 'Deg-malig-1':False, 'Inv-nodes 0-2': True, 'Inv-nodes 3-5': False, 'Inv-nodes 6-8': False},
    {'ge40':False, 'it40':False, 'premeno':True, 'Deg-malig-3':False, 'Deg-malig-2':False, 'Deg-malig-1':True, 'Inv-nodes 0-2': True, 'Inv-nodes 3-5': False, 'Inv-nodes 6-8': False}    
]


R1_condition = ['Deg-malig-3', 'NOT it40']
R2_condition = ['Deg-malig-3', 'NOT it40']
R3_condition = ['Inv-nodes 0-2']

def evaluate_condition(observation, R1_condition, R2_condition, R3_condition):
    truth_value_of_condition = True
    for feature in observation:
        if feature in R1_condition and observation[feature] == False:
            truth_value_of_condition = False
            break
        if feature in R2_condition and observation[feature] == False:
            truth_value_of_condition = False
            break
        if feature in R3_condition and observation[feature] == True:
            truth_value_of_condition = False
            break
    return truth_value_of_condition



for j in patient:
    print(evaluate_condition(j, R1_condition, R2_condition, R3_condition))
    




True
False
True
False
False
False
